In [7]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO

In [8]:
URL = 'https://www.opensecrets.org/races/summary.csv?cycle=2020&id=TN07'
response = requests.get(URL)
print(type(response))
if response.status_code == requests.codes.ok:
    print('Request is okay!')
else:
    response.raise_for_status()
TN_district7 = pd.read_csv(StringIO(response.text), sep=',')
TN_district7

<class 'requests.models.Response'>
Request is okay!


cid           FirstLastP       Rcpts      Spent      PACs     Indivs  \
0  N00041873       Mark Green (R)  1194960.47  935486.67  171900.0  819151.42   
1  N00045536   Kiran Sreepada (D)   206644.28  207190.98    4000.0  202644.28   
2  N00047077     Ronald Brown (I)     1750.00       0.00       0.0    1750.00   
3  N00046592  Scott Vieira Jr (I)      655.47    1048.51      10.0      45.00   
4  N00045535   Benjamin Estes (3)        0.00       0.00       0.0       0.00   

   Cand      Other    EndCash   LgIndivs  ...  Result CRPICO      State  \
0   0.0  203909.05  287888.55  819151.42  ...       W      I  Tennessee   
1   0.0       0.00       0.00  179129.75  ...       L      C  Tennessee   
2   0.0       0.00    9006.00     300.00  ...       L      C  Tennessee   
3  35.0     565.47    -196.52       0.00  ...       L      C  Tennessee   
4   0.0       0.00       0.00       0.00  ...              C  Tennessee   

   IncCID Incumbent                primarydate DistIDCurr  capeye  sort  \
0     NaN       NaN  2020-08-06 00:00:00 +0000       TN07       0     1   
1     NaN       NaN  2020-08-06 00:00:00 +0000                  0     2   
2     NaN       NaN  2020-08-06 00:00:00 +0000                  0     2   
3     NaN       NaN  2020-08-06 00:00:00 +0000                  0     2   
4     NaN       NaN  2020-08-06 00:00:00 +0000        NaN       0     2   

  SmLgIndivsNote  
0              N  
1              N  
2              N  
3              N  
4              N  

[5 rows x 24 columns]

In [9]:
def ensure_two_digits(num):
    """Ensures an integer is represented by two digits, padding with '0' if necessary."""
    return str(num).zfill(2)
    
def retrieve_2020_state_district_data(state: str, district: int, state_abbreviations=False):
    #need some kind of dictionary that will take state name if state_abbreviations=False
    state_abr_dict = {'Tennessee':'TN'}
    base_url = 'https://www.opensecrets.org/races/summary.csv?cycle=2020&id='
    district_num = ensure_two_digits(district)
    if state_abbreviations:
        state_district_url = base_url+state+district_num
    else:
        state_district_url = base_url+state_abr_dict[state]+district_num
    response = requests.get(state_district_url)
    if response.status_code != requests.codes.ok:
        response.raise_for_status()
        
    df = pd.read_csv(StringIO(response.text), sep=',')
    return pd.read_csv(StringIO(response.text), sep=',')



In [11]:
URL = 'https://en.wikipedia.org/wiki/List_of_U.S._state_and_territory_abbreviations'
states_df = pd.read_html(URL)[1]
states_df.columns = states_df.columns.map(lambda x: x[1])
states_df = (
    states_df
    .reset_index()
    .drop(columns = ['index', 'Status of region', 'Unnamed: 2_level_1', 'Unnamed: 4_level_1', 'Unnamed: 5_level_1', 'Unnamed: 6_level_1', 'GPO', 'AP', 'Other abbreviations'])
    .dropna()
    .rename(columns = {'Name': 'State', 'Unnamed: 3_level_1': 'Abbreviation'})
    .drop(0).reset_index(drop=True)
)
states_df

State Abbreviation
0                       Alabama           AL
1                        Alaska           AK
2                       Arizona           AZ
3                      Arkansas           AR
4                    California           CA
5                      Colorado           CO
6                   Connecticut           CT
7                      Delaware           DE
8          District of Columbia           DC
9                       Florida           FL
10                      Georgia           GA
11                       Hawaii           HI
12                        Idaho           ID
13                     Illinois           IL
14                      Indiana           IN
15                         Iowa           IA
16                       Kansas           KS
17                     Kentucky           KY
18                    Louisiana           LA
19                        Maine           ME
20                     Maryland           MD
21                Massachusetts           MA
22                     Michigan           MI
23                    Minnesota           MN
24                  Mississippi           MS
25                     Missouri           MO
26                      Montana           MT
27                     Nebraska           NE
28                       Nevada           NV
29                New Hampshire           NH
30                   New Jersey           NJ
31                   New Mexico           NM
32                     New York           NY
33               North Carolina           NC
34                 North Dakota           ND
35                         Ohio           OH
36                     Oklahoma           OK
37                       Oregon           OR
38                 Pennsylvania           PA
39                 Rhode Island           RI
40               South Carolina           SC
41                 South Dakota           SD
42                    Tennessee           TN
43                        Texas           TX
44                         Utah           UT
45                      Vermont           VT
46                     Virginia           VA
47                   Washington           WA
48                West Virginia           WV
49                    Wisconsin           WI
50                      Wyoming           WY
51               American Samoa           AS
52                         Guam           GU
53     Northern Mariana Islands           MP
54                  Puerto Rico           PR
55          U.S. Virgin Islands           VI
56  U.S. Minor Outlying Islands           UM
57             Marshall Islands           MH
58                   Micronesia           FM
59                        Palau           PW

In [12]:
retrieve_2020_state_district_data()

SyntaxError: invalid character '‘' (U+2018) (3869966078.py, line 1)

In [14]:
URL = 'https://en.wikipedia.org/wiki/2020_United_States_House_of_Representatives_elections'
response = requests.get(URL)
if response.status_code == requests.codes.ok:
    soup = BeautifulSoup(response.text, features="html.parser")
else:
    response.raise_for_status()

tables_html = str(soup.find_all('table', attrs={'class' : 'wikitable'}))
all_states_df = pd.read_html(StringIO(str(tables_html)))[1].fillna('-')
all_states_df.columns = all_states_df.columns.map(lambda x: x[1])
all_states_df = all_states_df.drop(columns=['Seats', 'Change'])
all_states_df.head(3)

state_seats_abbs = pd.merge(left=all_states_df, right=states_df, on='State')

In [15]:
state_seats_abbs

State  Total seats Abbreviation
0          Alabama            7           AL
1           Alaska            1           AK
2          Arizona            9           AZ
3         Arkansas            4           AR
4       California           53           CA
5         Colorado            7           CO
6      Connecticut            5           CT
7         Delaware            1           DE
8          Florida           27           FL
9          Georgia           14           GA
10          Hawaii            2           HI
11           Idaho            2           ID
12        Illinois           18           IL
13         Indiana            9           IN
14            Iowa            4           IA
15          Kansas            4           KS
16        Kentucky            6           KY
17       Louisiana            6           LA
18           Maine            2           ME
19        Maryland            8           MD
20   Massachusetts            9           MA
21        Michigan           14           MI
22       Minnesota            8           MN
23     Mississippi            4           MS
24        Missouri            8           MO
25         Montana            1           MT
26        Nebraska            3           NE
27          Nevada            4           NV
28   New Hampshire            2           NH
29      New Jersey           12           NJ
30      New Mexico            3           NM
31        New York           27           NY
32  North Carolina           13           NC
33    North Dakota            1           ND
34            Ohio           16           OH
35        Oklahoma            5           OK
36          Oregon            5           OR
37    Pennsylvania           18           PA
38    Rhode Island            2           RI
39  South Carolina            7           SC
40    South Dakota            1           SD
41       Tennessee            9           TN
42           Texas           36           TX
43            Utah            4           UT
44         Vermont            1           VT
45        Virginia           11           VA
46      Washington           10           WA
47   West Virginia            3           WV
48       Wisconsin            8           WI
49         Wyoming            1           WY